###Unidade 5  - Recuperação da Informação
Aluno(a): Tainá da Silva Lima
<br>
DRE: 116165607
<br><br>
Aluno(a): Rafael Pais Cardoso
<br>
DRE: 116140788
<br><br>
Professor(a): Giseli Lopes
<br>
Período: 2020.4 PLE

# Importando as bibliotecas necessárias aos exercícios realizados nesse trabalho

In [ ]:
import numpy as np
import math
from prettytable import PrettyTable

# Funções

Pré-processamento da coleção

In [ ]:
# M             --> Vetor coluna onde cada linha representa o texto de um documento (matriz Nx1).
# separadores   --> Vetor linha de caracteres (matriz 1xNc), onde o elemento em cada coluna representa um separador a ser usado na tokenização dos documentos.
# stopwords     --> Vetor linha de strings (matriz 1xNs), onde o elemento em cada coluna armazena uma stopword.
# Essa função aplica as etapas de pré-processamento nos documentos.
def normalizacaoTokenizacao(M, separadores, stopwords):

  matriz = []

  for index in range(0, len(M)):

    # Recebe o texto de um novo documento a cada iteração.
    documentoTemporario = M[index][0]

    # Remoção dos delimitadores e normalização.
    for delimitador in separadores:
      documentoTemporario = documentoTemporario.lower().replace(delimitador, " ")

    # Obtenção dos possíveis tokens
    tokens = documentoTemporario.split()

    # Remoção das stopwords e armazenamento dos tokens em uma matriz Nx1 (cada linha representa um documento e a coluna apresenta a lista de tokens extraídos do documento).
    matriz.append(list(filter(lambda a: a not in stopwords, tokens)))
    
  return matriz

Pré-processamento da consulta

In [ ]:
# consulta      --> String contendo os termos da consulta.
# separadores   --> Vetor linha de caracteres (matriz 1xNc), onde o elemento em cada coluna representa um separador a ser usado na tokenização dos documentos.
# stopwords     --> Vetor linha de strings (matriz 1xNs), onde o elemento em cada coluna armazena uma stopword.
# Essa função aplica as etapas de pré-processamento na consulta.
def tratamentoConsulta(consulta, separadores, stopwords):

  consultaTratada = consulta
  for delimitador in separadores:
      consultaTratada = consultaTratada.lower().replace(delimitador, " ")
  
  consultaTratada = consultaTratada.split()
  
  consultaTratada = list(filter(lambda a: a not in stopwords, consultaTratada))
  
  return consultaTratada

Determinação da frequência do termo no documento

In [ ]:
# termo     --> String contendo um único elemento.
# documento --> Vetor de string (matriz 1x1), que representa o texto de um único documento da coleção.
# Calcula a frequência de um determinado termo em um documento específico.
def freqDoc(termo, documento):
  return documento.count(termo)

Determinação da frequência do termo na coleção

In [ ]:
# termo   --> String contendo um único elemento.
# colecao --> Vetor de string (matriz 1x1), que representa o texto de um único documento da coleção.
# Calcula a frequência de um determinado termo em um documento específico.
def freqCol(termo, colecao):
  contaTermo = 0
  for documento in colecao:
    if(termo in documento):
      contaTermo+= 1
  return contaTermo

Cálculo TF-IDF

Aplicamos a fórmula abaixo para calcular o peso para os termos dos documentos e para a consulta 

$\left ( 1+\log f_{i,j}\right ) * \left ( \log\frac{N}{^{N_{i}}} \right )$

In [ ]:
# colecao --> Vetor coluna onde cada linha representa o texto de um documento que foi pré-processado, ou seja, foi aplicado normalização e tokenização.
# Esta função realiza a ponderação TF-IDF nos termos dos documentos e retorna um dicionário de dicionário em que a chave do dicionário mais externo se 
# refere ao documento e o valor associado à chave é um outro dicionário em que a chave é o termo e o valor associado é o seu peso.
def ponderacaoTF_IDF(colecao):

  termoLista      = []
  dicionarioPeso  = {}

  # Insere os termos dos documentos na lista "termoLista", mas sem repetição
  for documentos in colecao:
    for termo in documentos:
      if(termo not in termoLista):
        termoLista.append(termo)

  numeroDocumento = 0

  # Aplicação da fórmula do "Esquema de ponderação 3" do slide da aula
  for documento in colecao:
    dicionarioTemp = {}
    for termo in termoLista:
      peso = 0
      if(termo in documento):
        peso = (1 + math.log2(freqDoc(termo, documento))) * (math.log2(len(colecao)/freqCol(termo, colecao)))
      if(termo not in dicionarioTemp.keys()):
        dicionarioTemp[termo] = peso

    dicionarioPeso[numeroDocumento] = dicionarioTemp
    numeroDocumento+= 1

  return dicionarioPeso

In [ ]:
# colecao         --> Vetor coluna onde cada linha representa o texto de um documento que foi pré-processado, ou seja, foi aplicado normalização e tokenização.
# consultaTratada --> Lista com os termos da consulta pré-processados
# Realiza a ponderação TF-IDF nos termos da consulta e retorna um dicionário em que as chaves são os termos e os valores são os respectivos pesos.
def ponderacaoTF_IDF2(colecao, consultaTratada):

  termoLista = []

  # Insere os termos dos documentos na lista "termoLista", mas sem repetição
  for documentos in colecao:
    for termo in documentos:
      if(termo not in termoLista):
        termoLista.append(termo)

  numeroDocumento = 0

  dicionarioTemp  = {}

  # Aplicação da fórmula do "Esquema de ponderação 3" do slide da aula
  for termo in termoLista:
    peso = 0
    if(termo in consultaTratada):
      peso = (1 + math.log2( freqDoc(termo, consultaTratada) ) ) * (math.log2( len(colecao) / freqCol(termo, colecao) ) )
    if(termo not in dicionarioTemp.keys()):
      dicionarioTemp[termo] = peso

  return dicionarioTemp


# vetorConsulta --> consulta representada como vetores de termos com pesos associados
# vetorDoc      --> documento representado como vetores de termos com pesos associados
# Cálculo do cosseno do ângulo formado com o vetor da consulta e o vetor do documento
def sim(vetorDoc, vetorConsulta):
  somatorio = 0

  for i,j in zip(vetorDoc.keys(),range(len(vetorDoc))):
    somatorio += vetorDoc[i] * vetorConsulta[j]

  moduloVetorDoc      = np.linalg.norm(np.array(list(vetorDoc.values())))
  moduloVetorConsulta = np.linalg.norm(np.array(vetorConsulta))

  return somatorio/(moduloVetorDoc*moduloVetorConsulta)


# dicPesosDoc      --> dicionário obtido da ponderação TF-IDF aplicada nos documentos
# dicPesosConsulta --> dicionário obtido da ponderação TF-IDF aplicada na consulta
# Aplica o ranqueamento dos documentos
def rank(dicPesosDoc, dicPesosConsulta):
  
  dicRank = {}

  for key in dicPesosDoc.keys():
    rankDict = sim(dicPesosDoc[key], list(dicPesosConsulta.values()))
    dicRank[key] = rankDict
  
  return dicRank

# Exemplo 1


Coleção, stopwords, consulta e separadores

In [ ]:
# Conjunto de documentos
M1            = [["O peã e o caval são pec de xadrez. O caval é o melhor do jog."],
                ["A jog envolv a torr, o peã e o rei."],
                ["O peã lac o boi"],
                ["Caval de rodei!"],
                ["Polic o jog no xadrez."]]

# Lista de stopwords
stopwords1    = ["a", "o", "e", "é", "de", "do", "no", "são"]

# String contendo os termos da consulta
q1            = "xadrez peã caval torr"

# Separadores para a tokenizacao
separadores1  = [" ",",",".","!","?"] 

Aplicação do pré-processamento na coleção e na consulta e aplicação do TF-IDF na coleção

In [ ]:
resultadoEx1            = normalizacaoTokenizacao(M1, separadores1, stopwords1)
print("Resultado da normalização e tokenização:")
print(resultadoEx1)
print("\n")

consultaTratadaEx1      = tratamentoConsulta(q1, separadores1, stopwords1)
print("Resultado do tratamento da consulta:")
print(consultaTratadaEx1)
print('\n')

ponderacaoDocumentosEx1 = ponderacaoTF_IDF(resultadoEx1)
print("Resultado da ponderação TF-IDF para os termos da coleção:")
print(ponderacaoDocumentosEx1)
print('\n')

var1                    = ponderacaoTF_IDF2(resultadoEx1, consultaTratadaEx1)
dicRank1                = rank(ponderacaoTF_IDF(resultadoEx1), var1)
listRank1               = sorted(dicRank1.items(), key=lambda x: x[1], reverse=True)

print("Ranking:")

tableRank1              = PrettyTable()

column_names1           = ["Posição", "Documento", "Rank"] 
#print(listRank1)

tableRank1.add_column(column_names1[0], [i+1 for i in range(len(listRank1))])
tableRank1.add_column(column_names1[1], [M1[document[0]][0] for document in listRank1])
tableRank1.add_column(column_names1[2], [str(document[1]) for document in listRank1])

print(tableRank1)

Resultado da normalização e tokenização:
[['peã', 'caval', 'pec', 'xadrez', 'caval', 'melhor', 'jog'], ['jog', 'envolv', 'torr', 'peã', 'rei'], ['peã', 'lac', 'boi'], ['caval', 'rodei'], ['polic', 'jog', 'xadrez']]


Resultado do tratamento da consulta:
['xadrez', 'peã', 'caval', 'torr']


Resultado da ponderação TF-IDF para os termos da coleção:
{0: {'peã': 0.7369655941662062, 'caval': 2.643856189774725, 'pec': 2.321928094887362, 'xadrez': 1.3219280948873624, 'melhor': 2.321928094887362, 'jog': 0.7369655941662062, 'envolv': 0, 'torr': 0, 'rei': 0, 'lac': 0, 'boi': 0, 'rodei': 0, 'polic': 0}, 1: {'peã': 0.7369655941662062, 'caval': 0, 'pec': 0, 'xadrez': 0, 'melhor': 0, 'jog': 0.7369655941662062, 'envolv': 2.321928094887362, 'torr': 2.321928094887362, 'rei': 2.321928094887362, 'lac': 0, 'boi': 0, 'rodei': 0, 'polic': 0}, 2: {'peã': 0.7369655941662062, 'caval': 0, 'pec': 0, 'xadrez': 0, 'melhor': 0, 'jog': 0, 'envolv': 0, 'torr': 0, 'rei': 0, 'lac': 2.321928094887362, 'boi': 2.321928094

# Exemplo 2


Coleção, stopwords, consulta e separadores

In [ ]:
# Conjunto de documentos
M2              = [["Parasita é o grande vencedor do Oscar 2020, com quatro prêmios"],
                  ["Green Book, Roma e Bohemian Rhapsody são os principais vencedores do Oscar 2019"],
                  ["Oscar 2020: Confira lista completa de vencedores. Parasita e 1917 foram os grandes vencedores da noite"],
                  ["Em boa fase, Oscar sonha em jogar a Copa do Mundo da Rússia"],
                  ["Conheça os indicados ao Oscar 2020; Cerimônia de premiação acontece em fevereiro"],
                  ["Oscar Schmidt receberá Troféu no Prêmio Brasil Olímpico 2019. Jogador de basquete com mais pontos em Jogos Olímpicos."],
                  ["Seleção brasileira vai observar de 35 a 40 jogadores para definir lista da Copa América"],
                  ["Oscar 2020: saiba como é a escolha dos jurados e como eles votam"],
                  ["Bem, Amigos! discute lista da Seleção, e Galvão dá recado a Tite: Cadê o Luan?"],
                  ["IFAL-Maceió convoca aprovados em lista de espera do SISU para chamada oral"],
                  ["Arrascaeta e Matías Viña são convocados pelo Uruguai para eliminatórias da Copa. Além deles, há outros destaques na lista."],
                  ["Oscar do Vinho: confira os rótulos de destaque da safra 2018"],
                  ["Parasita é o vencedor da Palma de Ouro no Festival de Cannes"],
                  ["Estatísticas. Brasileirão Série A: Os artilheiros e garçons da temporada 2020"],
                  ["Setembro chegou! Confira o calendário da temporada 2020/2021 do futebol europeu"]]

# Lista de stopwords
stopwords2     = ["a", "o", "e", "é", "de", "do", "da", "no", "na", "são", "dos", "com", "como", "eles", "em", "os", "ao", "para", "pelo"] 

# String contendo os termos da consulta
q2             = "parasita vencedor green"

# Separadores para a tokenizacao
separadores2   = [" ",",",".","!","?",":","/","\\",";" ] 

Aplicação do pré-processamento na coleção e na consulta e aplicação do TF-IDF na coleção

In [ ]:
resultadoEx2            = normalizacaoTokenizacao(M2, separadores2, stopwords2)
print("Resultado da normalização e tokenização:")
print(resultadoEx2)
print('\n')

consultaTratadaEx2      = tratamentoConsulta(q2, separadores2, stopwords2)
print("Resultado do tratamento da consulta:")
print(consultaTratadaEx2)
print('\n')

ponderacaoDocumentosEx2 = ponderacaoTF_IDF(resultadoEx2)
print("Resultado da ponderação TF-IDF para os termos da coleção:")
print(ponderacaoDocumentosEx2)
print('\n')

var2                    = ponderacaoTF_IDF2(resultadoEx2, consultaTratadaEx2)
dicRank2                = rank(ponderacaoTF_IDF(resultadoEx2), var2)
listRank2               = sorted(dicRank2.items(), key=lambda x: x[1], reverse=True)

print("Ranking:")

tableRank2              = PrettyTable()

column_names2           = ["Posição", "Documento", "Rank"] 
#print(listRank2)

tableRank2.add_column(column_names2[0], [i+1 for i in range(len(listRank2))])
tableRank2.add_column(column_names2[1], [M2[document[0]][0] for document in listRank2])
tableRank2.add_column(column_names2[2], [str(document[1]) for document in listRank2])

print(tableRank2)

Resultado da normalização e tokenização:
[['parasita', 'grande', 'vencedor', 'oscar', '2020', 'quatro', 'prêmios'], ['green', 'book', 'roma', 'bohemian', 'rhapsody', 'principais', 'vencedores', 'oscar', '2019'], ['oscar', '2020', 'confira', 'lista', 'completa', 'vencedores', 'parasita', '1917', 'foram', 'grandes', 'vencedores', 'noite'], ['boa', 'fase', 'oscar', 'sonha', 'jogar', 'copa', 'mundo', 'rússia'], ['conheça', 'indicados', 'oscar', '2020', 'cerimônia', 'premiação', 'acontece', 'fevereiro'], ['oscar', 'schmidt', 'receberá', 'troféu', 'prêmio', 'brasil', 'olímpico', '2019', 'jogador', 'basquete', 'mais', 'pontos', 'jogos', 'olímpicos'], ['seleção', 'brasileira', 'vai', 'observar', '35', '40', 'jogadores', 'definir', 'lista', 'copa', 'américa'], ['oscar', '2020', 'saiba', 'escolha', 'jurados', 'votam'], ['bem', 'amigos', 'discute', 'lista', 'seleção', 'galvão', 'dá', 'recado', 'tite', 'cadê', 'luan'], ['ifal-maceió', 'convoca', 'aprovados', 'lista', 'espera', 'sisu', 'chamada', '

# Exemplo 3

Coleção, stopwords, consulta e separadores

In [ ]:
# Conjunto de documentos
M3              = [["To do is to be. To be is to do."],
                  ["To be or not to be. I am what I am."],
                  ["I think therefore I am. Do be do be do."],
                  ["Do do do, da da da. Let it be, let it be."]]

# Lista de stopwords
stopwords3     = [] 


# String contendo os termos da consulta
q3             = "is it"

# Separadores para a tokenizacao
separadores3   = [" ",",",".","!","?",":","/","\\",";" ]           

Aplicação do pré-processamento na coleção e na consulta e aplicação do TF-IDF na coleção

In [ ]:
resultadoEx3            = normalizacaoTokenizacao(M3, separadores3, stopwords3)
print("Resultado da normalização e tokenização:")
print(resultadoEx3)
print('\n')

consultaTratadaEx3      = tratamentoConsulta(q3, separadores3, stopwords3)
print("Resultado do tratamento da consulta:")
print(consultaTratadaEx3)
print('\n')

ponderacaoDocumentosEx3 = ponderacaoTF_IDF(resultadoEx3)
print("Resultado da ponderação TF-IDF para os termos da coleção:")
print(ponderacaoDocumentosEx3)
print('\n')

var3                    = ponderacaoTF_IDF2(resultadoEx3, consultaTratadaEx3)
dicRank3                = rank(ponderacaoTF_IDF(resultadoEx3), var3)
listRank3               = sorted(dicRank3.items(), key=lambda x: x[1], reverse=True)

print("Ranking:")

tableRank3              = PrettyTable()

column_names3           = ["Posição", "Documento", "Rank"] 
#print(listRank3)

tableRank3.add_column(column_names3[0], [i+1 for i in range(len(listRank3))])
tableRank3.add_column(column_names3[1], [M3[document[0]][0] for document in listRank3])
tableRank3.add_column(column_names3[2], [str(document[1]) for document in listRank3])

print(tableRank3)

Resultado da normalização e tokenização:
[['to', 'do', 'is', 'to', 'be', 'to', 'be', 'is', 'to', 'do'], ['to', 'be', 'or', 'not', 'to', 'be', 'i', 'am', 'what', 'i', 'am'], ['i', 'think', 'therefore', 'i', 'am', 'do', 'be', 'do', 'be', 'do'], ['do', 'do', 'do', 'da', 'da', 'da', 'let', 'it', 'be', 'let', 'it', 'be']]


Resultado do tratamento da consulta:
['is', 'it']


Resultado da ponderação TF-IDF para os termos da coleção:
{0: {'to': 3.0, 'do': 0.8300749985576874, 'is': 4.0, 'be': 0.0, 'or': 0, 'not': 0, 'i': 0, 'am': 0, 'what': 0, 'think': 0, 'therefore': 0, 'da': 0, 'let': 0, 'it': 0}, 1: {'to': 2.0, 'do': 0, 'is': 0, 'be': 0.0, 'or': 2.0, 'not': 2.0, 'i': 2.0, 'am': 2.0, 'what': 2.0, 'think': 0, 'therefore': 0, 'da': 0, 'let': 0, 'it': 0}, 2: {'to': 0, 'do': 1.0728563720288948, 'is': 0, 'be': 0.0, 'or': 0, 'not': 0, 'i': 2.0, 'am': 1.0, 'what': 0, 'think': 2.0, 'therefore': 2.0, 'da': 0, 'let': 0, 'it': 0}, 3: {'to': 0, 'do': 1.0728563720288948, 'is': 0, 'be': 0.0, 'or': 0, 'not